In [ ]:
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
!unzip -q tiny-imagenet-200.zip

--2023-01-07 11:03:06--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.68.10
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  18.1MB/s    in 23s     

2023-01-07 11:03:29 (10.1 MB/s) - ‘tiny-imagenet-200.zip’ saved [248100043/248100043]



In [ ]:
#Importing the required libraries
import numpy as np
from skimage import color
import os, time, shutil, argparse
import matplotlib.pyplot as plt
from skimage.color import rgb2lab, rgb2gray, lab2rgb
import matplotlib.image as mpimg

import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import datasets, transforms
import torchvision.models as models

# Check if GPU is available
use_gpu = torch.cuda.is_available()

In [ ]:
#A folder for loading image and converting them to LAB color space
class LABImageFolder(datasets.ImageFolder):
  '''Custom images folder, which converts images to grayscale before loading'''
  def __getitem__(self, index):
    path, target = self.imgs[index]
    img = self.loader(path)

    img_lab = rgb2lab(img)

    img_l = img_lab[:,:,0:1]
    # shape of img_l is (64,64,1)

    img_ab = img_lab[:, :, 1:3]
    # shape of img_ab is (64,64,2).

    tens_l = torch.from_numpy(img_l.transpose((2, 0, 1))).float()

    # The next lines convert the "ab" channels to a PyTorch tensor.
    tens_ab = torch.from_numpy(img_ab.transpose((2, 0, 1))).float()

    return tens_l, tens_ab

In [ ]:
train_imagefolder = LABImageFolder('/content/tiny-imagenet-200/train')  # gets all images from training folder & turns them into 2 tensors/image!
train_loader = torch.utils.data.DataLoader(train_imagefolder, batch_size=64, shuffle=True)

In [ ]:
val_imagefolder = LABImageFolder('/content/tiny-imagenet-200/val')
val_loader = torch.utils.data.DataLoader(val_imagefolder, batch_size=64, shuffle=False)

In [ ]:
# BaseColor is a custom Class that inherits from Class nn.Module
# The CNN model, we use, is a subclass of the BaseColor Class!

class BaseColor(nn.Module):
	def __init__(self):
		super(BaseColor, self).__init__()

		self.l_cent = 50.
		self.l_norm = 100.
		self.ab_norm = 110.

	def normalize_l(self, in_l):
		return (in_l-self.l_cent)/self.l_norm

	def unnormalize_l(self, in_l):
		return in_l*self.l_norm + self.l_cent

	def normalize_ab(self, in_ab):
		return in_ab/self.ab_norm

	def unnormalize_ab(self, in_ab):
		return in_ab*self.ab_norm

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from IPython import embed

class ECCVGenerator(BaseColor):
    def __init__(self, norm_layer=nn.BatchNorm2d):   # Why this norm_layer==nn.BatchNorm2d? it is used in several of the models
        super(ECCVGenerator, self).__init__()

        model1=[nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=True),]  # ok, we go from 1 layer to 64 layers. Where are the filters defined?
        model1+=[nn.ReLU(True),]
        model1+=[nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=1, bias=True),]
        model1+=[nn.ReLU(True),]
        model1+=[norm_layer(64),]

        model2=[nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=True),]
        model2+=[nn.ReLU(True),]
        model2+=[nn.Conv2d(128, 128, kernel_size=3, stride=2, padding=1, bias=True),]
        model2+=[nn.ReLU(True),]
        model2+=[norm_layer(128),]

        model3=[nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1, bias=True),]
        model3+=[nn.ReLU(True),]
        model3+=[nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True),]
        model3+=[nn.ReLU(True),]
        model3+=[nn.Conv2d(256, 256, kernel_size=3, stride=2, padding=1, bias=True),]
        model3+=[nn.ReLU(True),]
        model3+=[norm_layer(256),]

        model4=[nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model4+=[nn.ReLU(True),]
        model4+=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model4+=[nn.ReLU(True),]
        model4+=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model4+=[nn.ReLU(True),]
        model4+=[norm_layer(512),]

        model5=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model5+=[nn.ReLU(True),]
        model5+=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model5+=[nn.ReLU(True),]
        model5+=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model5+=[nn.ReLU(True),]
        model5+=[norm_layer(512),]

        model6=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model6+=[nn.ReLU(True),]
        model6+=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model6+=[nn.ReLU(True),]
        model6+=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model6+=[nn.ReLU(True),]
        model6+=[norm_layer(512),]

        model7=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model7+=[nn.ReLU(True),]
        model7+=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model7+=[nn.ReLU(True),]
        model7+=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model7+=[nn.ReLU(True),]
        model7+=[norm_layer(512),]

        model8=[nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=True),]
        model8+=[nn.ReLU(True),]
        model8+=[nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True),]
        model8+=[nn.ReLU(True),]
        model8+=[nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True),]
        model8+=[nn.ReLU(True),] # <- original, normally followed by a 256 -> 313 ... and then softmax!
        # The relu is outputting values from 0 to infinity!


        # model8+=[nn.Conv2d(256, 313, kernel_size=1, stride=1, padding=0, bias=True),] # <- changed by Dirk, actually not doing this thing at all
        # model8+=[nn.Conv2d(256, 313, kernel_size=1, stride=1, padding=0, bias=True),]  <- original
        # What would be the range of values after this conv2d? Before moving to softmax?
        # kernel_size = 1 --> no filter of feature extractor!!!
        # stride = 1 --> horizontal + vertical size remains the same!!
        # This layer was only to go to 313 layers, without changing spatial dimensions!
        # We don

        self.model1 = nn.Sequential(*model1)
        self.model2 = nn.Sequential(*model2)
        self.model3 = nn.Sequential(*model3)
        self.model4 = nn.Sequential(*model4)
        self.model5 = nn.Sequential(*model5)
        self.model6 = nn.Sequential(*model6)
        self.model7 = nn.Sequential(*model7)
        self.model8 = nn.Sequential(*model8)

        # self.softmax = nn.Softmax(dim=1)  # what does it mean to softmax over dimension 1? Dimension 1 is the number of layers in the tensor!
        # self.model_out = nn.Conv2d(313, 2, kernel_size=1, padding=0, dilation=1, stride=1, bias=False)  <- original code
        self.model_out = nn.Conv2d(256, 2, kernel_size=1, padding=0, dilation=1, stride=1, bias=False) # <- changed by Dirk
        self.upsample4 = nn.Upsample(scale_factor=4, mode='bilinear')

    def forward(self, input_l):
        conv1_2 = self.model1(self.normalize_l(input_l))   # normalizing "l" <- one of the functions/methods in base_color.py
        # Attention we are normalizing the values here, so there is a problem if they already have been normalised!!!
        conv2_2 = self.model2(conv1_2)
        conv3_3 = self.model3(conv2_2)
        conv4_3 = self.model4(conv3_3)
        # conv5_3 = self.model5(conv4_3)
        # conv6_3 = self.model6(conv5_3)
        conv7_3 = self.model7(conv4_3)   #conv6_3
        conv8_3 = self.model8(conv7_3)
        # out_reg = self.model_out(self.softmax(conv8_3)) <- original
        out_reg = self.model_out(conv8_3) # <- changed by Dirk not using the softmax!!!!

        return self.unnormalize_ab(self.upsample4(out_reg))      # unnormalizing "ab" <- one of the functions/methods in base_color.py
        # to unnormalize_ab() it is required the values before unnormalizing are between -1 and 1 <-- tanh needed or not?
        # How was assured that values would be -1 to 1 before? Ah, there was a softmax, but followed by model_out (i.e. conv2d)!!!
        # Let's not go from 256 to 313, let's not softmax, let's not do a special model

def eccv16(pretrained=True):
	   model = ECCVGenerator()
	   if(pretrained):
		     import torch.utils.model_zoo as model_zoo
		     model.load_state_dict(model_zoo.load_url('https://colorizers.s3.us-east-2.amazonaws.com/colorization_release_v2-9b330a0b.pth',map_location='cpu',check_hash=True))
	   return model

In [ ]:
model=eccv16(pretrained=False)
if torch.cuda.is_available():
    model.cuda()
# print(model)

In [ ]:

criterion = nn.L1Loss()

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [ ]:
def train(train_loader, model, criterion, optimizer, epochs = 3):
  for i in range(epochs):

    epoch_loss = 0
    print("Epoch:", i+1)

    model.train()

    for i_batch, (input_l_batch, input_ab_batch) in enumerate(train_loader):             # i_batch is the batch number

      if use_gpu: input_l_batch, input_ab_batch = input_l_batch.cuda(), input_ab_batch.cuda() # Load them in GPU, if available

      optimizer.zero_grad()

      output_ab = model(input_l_batch)

      # Determine loss function
      loss = criterion(output_ab, input_ab_batch)

      # Back propagation
      loss.backward()

      # Updates weights
      optimizer.step()

      epoch_loss += loss.item()

    # Print loss Every Epoch
    print("Train Loss: ", epoch_loss / len(train_imagefolder))
  # https://pytorch.org/tutorials/beginner/saving_loading_models.html#saving-loading-model-for-inference
  torch.save(model,'pretrained_regression_model.pth')

In [ ]:
train(train_loader=train_loader, model=model, criterion=criterion, optimizer=optimizer, epochs=3)

Epoch: 1


KeyboardInterrupt: ignored

In [ ]:
def validate(val_loader, model, criterion, optimizer, epochs = 3):
  for i in range(epochs):

    val_total_loss = 0
    print("Epoch:", i+1)

    model.eval()

    for i_batch, (input_l_batch, input_ab_batch) in enumerate(val_loader):# i_batch is the batch number

      if use_gpu: input_l_batch, input_ab_batch = input_l_batch.cuda(), input_ab_batch.cuda() # Load them in GPU, if available

      with torch.no_grad():
        output_ab = model(input_l_batch)
        # Determine loss function
        loss = criterion(output_ab, input_ab_batch)

      val_total_loss += loss.item()

    # Print loss Every Epoch
    print("Validation Loss: ", val_total_loss / len(val_imagefolder))

In [ ]:
validate(val_loader=val_loader, model=model, criterion=criterion, optimizer=optimizer, epochs=2)

In [ ]:
pretrained_regression_model = torch.load('pretrained_regression_model.pth')
if torch.cuda.is_available():
     pretrained_regression_model.cuda()

In [ ]:
def postprocess_tens(tens_l, out_ab):
	out_lab = torch.cat((tens_l, out_ab), dim=1)
	return color.lab2rgb(out_lab.data.cpu().numpy()[0,...].transpose((1,2,0)))

In [ ]:
for i in range (10000,10200):
  tens_l, tens_ab = train_imagefolder[i]
  if use_gpu:
    tens_l, tens_ab = tens_l.cuda(), tens_ab.cuda()
  tens_l = tens_l.unsqueeze(0) # needed because model takes into account batch dimension
  tens_ab = tens_ab.unsqueeze(0) # needed because model takes into account batch dimension
  tens_ab_out = model(tens_l)
  print(tens_ab_out.size())
  print(tens_ab.size())
  image = postprocess_tens(tens_l, tens_ab_out)
  plt.imshow(image)
  plt.show()